In [ ]:
import pandas as pd
from darts import TimeSeries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Read a pandas DataFrame
path = "/Users/nico_marais/code/Liedjieboer/forward_farming/clean_data/clean_sheff_data.csv"
df = pd.read_csv(path, delimiter=",")

In [ ]:
df.head()

In [ ]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, "Unnamed: 0", "rain(mm)")

# Set aside the last 36 months as a validation series
train, val = series[:-36], series[-36:]

In [ ]:
from darts.models import ExponentialSmoothing

model_exp_smooth = ExponentialSmoothing()
model_exp_smooth.fit(train)
prediction_exp_smooth = model_exp_smooth.predict(360, num_samples=1000)

In [ ]:
import matplotlib.pyplot as plt

series.plot()
prediction_exp_smooth.plot(label="forecast", low_quantile=0.25, high_quantile=0.75)
plt.legend()

In [ ]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, "Unnamed: 0", "avgtemp(degC)")

# Set aside the last 36 months as a validation series
train, val = series[:-36], series[-36:]

from darts.models import NaiveSeasonal

model_naive = NaiveSeasonal()
model_naive.fit(train)
prediction_naive = model_naive.predict(36)

series.plot()
prediction_naive.plot(label="forecast", low_quantile=0.25, high_quantile=0.75)
plt.legend()

In [ ]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, "Unnamed: 0", "sun(hours)")

# Set aside the last 36 months as a validation series
train, val = series[:-36], series[-36:]

from darts.models import NaiveSeasonal

naive_model = NaiveSeasonal(K=12)
naive_model.fit(train)
naive_forecast = naive_model.predict(36)

series.plot(label="actual")
naive_forecast.plot(label="naive forecast (K=12)")
plt.xlim(left=1600)

In [ ]:

# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, "Unnamed: 0", "sun(hours)")

# Set aside the last 36 months as a validation series
train, val = series[:-36], series[-36:]

from darts.models import TFTModel

model_tft = TFTModel(
    input_chunk_length=12*3,
    output_chunk_length=12,
    add_relative_index=True
)
model_tft.fit(train)
prediction_tft = model_tft.predict(360, num_samples=1000)

series.plot()
prediction_tft.plot(label="forecast", low_quantile=0.25, high_quantile=0.75)
plt.legend()

In [ ]:
df.head()

In [ ]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, "Unnamed: 0", "avgtemp(degC)")
series.plot()

In [ ]:
seasonal_model = NaiveSeasonal(K=12)
seasonal_model.fit(train)
seasonal_forecast = seasonal_model.predict(36)

series.plot(label="actual")
seasonal_forecast.plot(label="naive forecast (K=12)")
plt.xlim(left=1600)

In [ ]:
from darts.models import NaiveDrift

drift_model = NaiveDrift()
drift_model.fit(train)
drift_forecast = drift_model.predict(36)

combined_forecast = drift_forecast + seasonal_forecast - train.last_value()

series.plot()
combined_forecast.plot(label="combined")
drift_forecast.plot(label="drift")
plt.xlim(left=1600)

In [ ]:
from darts.metrics import mape

print(
    "Mean absolute percentage error for the combined naive drift + seasonal: {:.2f}%.".format(
        mape(series, combined_forecast)
    )
)

In [ ]:
from darts.models import ExponentialSmoothing, TBATS, AutoARIMA, Theta


def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(val))
    print("model {} obtains MAPE: {:.2f}%".format(model, mape(val, forecast)))


eval_model(ExponentialSmoothing())


In [ ]:
eval_model(TBATS())


In [ ]:
eval_model(AutoARIMA())


In [ ]:
from darts.models import ARIMA
eval_model(ARIMA())


In [ ]:
from darts.models import BATS
eval_model(BATS())

In [1]:
from darts.models import BlockRNNModel
eval_model(BlockRNNModel(input_chunk_length=len(series), output_chunk_length=36))

/Users/nico_marais/.pyenv/versions/3.10.6/envs/forward_farming/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'eval_model' is not defined

In [ ]:
eval_model(Theta())

In [ ]:
from darts.models import CatBoostModel
eval_model(CatBoostModel(lags=36))

In [ ]:
from darts.models import Croston
eval_model(Croston())

In [ ]:
from darts.models import ExponentialSmoothing
eval_model(ExponentialSmoothing())

In [ ]:
from darts.models import FFT
eval_model(FFT())

In [ ]:
from darts.models import FourTheta
eval_model(FourTheta())

In [ ]:
from darts.models import KalmanForecaster
eval_model(KalmanForecaster())

In [ ]:
# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(df, "Unnamed: 0", "avgtemp(degC)")

# Set aside the last 36 months as a validation series
train, val = series[:-36], series[-36:]

In [ ]:
# Search for the best theta parameter, by trying 50 different values
thetas = 2 - np.linspace(-10, 10, 50)

best_mape = float("inf")
best_theta = 0

for theta in thetas:
    model = Theta(theta)
    model.fit(train)
    pred_theta = model.predict(len(val))
    res = mape(val, pred_theta)

    if res < best_mape:
        best_mape = res
        best_theta = theta

In [ ]:
best_theta_model = Theta(best_theta)
best_theta_model.fit(train)
pred_best_theta = best_theta_model.predict(len(val))

print(
    "The MAPE is: {:.2f}, with theta = {}.".format(
        mape(val, pred_best_theta), best_theta
    )
)

In [ ]:
train.plot(label="train")
val.plot(label="true")
pred_best_theta.plot(label="prediction")
plt.xlim(left=1600)